In [39]:
from PIL import Image
import numpy as np
from IPython.display import display
from pynq import Xlnk
from pynq import Overlay
CannyEdge_design = Overlay("./bitstream/CannyEdge.bit")
dma = CannyEdge_design.axi_dma_0
CannyEdge = CannyEdge_design.canny_edge_0
image_path = "images/paris.jpg"
original_image = Image.open(image_path)
display(original_image)

In [44]:
pic_width, pic_height = original_image.size
print("Image size: {}x{} pixels.".format(pic_width, pic_height))
original_image = original_image.convert('L') #gray scale the loaded image
input_array = np.array(original_image)
xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(pic_height, pic_width), 
                           dtype=np.uint8)
out_buffer = xlnk.cma_array(shape=(pic_height, pic_width), 
                            dtype=np.uint8)

Image size: 640x360 pixels.


In [48]:
np.copyto(in_buffer,input_array)
buf_image = Image.fromarray(in_buffer)
import time

start_time = time.time()


CannyEdge.write(0x14, pic_height) #rows
CannyEdge.write(0x1c, pic_width) #cols

#CannyEdge.write(0x7f6d722a58, 20) #Data signal of threshold1
#CannyEdge.write(0x7f6d722a5c, 100) #Data signal of threshold2

CannyEdge.write(0x24, 20) #Data signal of threshold1
CannyEdge.write(0x2c, 100) #Data signal of threshold2
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer) 
CannyEdge.write(0x00,0x81) # start
dma.sendchannel.wait()
dma.recvchannel.wait()
plt.imshow(out_buffer.reshape(pic_height, pic_width),cmap='gray')

end_time = time.time()

print("耗时：{}s".format(end_time - start_time))